In [121]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

import nltk
from nltk.corpus import wordnet as wn
import inflect

from keras.models import Sequential
from keras import layers
import numpy as np
from collections import Counter, defaultdict

from gensim.utils import tokenize
from itertools import groupby

from keras.models import Input, Model
from keras.layers import Dense, Dropout
from keras.layers import LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed

In [2]:
p = inflect.engine()

In [3]:
pairs = {}
for synset in wn.all_synsets('n'):
    word = synset.name().split('.', 1)[0]
    if not word in pairs:
        pairs[word] = p.plural(word)
len(pairs)

67176

In [4]:
with open('data/plurals.txt', 'w') as fout:
    for k in sorted(pairs):
        if '_' in k or '-' in k:
            continue
        if k.isdigit():
            continue
        fout.write('%s\t%s\n' % (k, pairs[k]))

In [5]:
p.plural('no')

'noes'

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [7]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [8]:
# Parameters for the model and dataset.
INVERT = True

In [108]:
questions = []
expected = []
seen = set()
#with open('data/en_de.txt') as fin:
with open('data/plurals.txt') as fin:
    for line in fin:
        en, de = line.strip().split('\t')
        questions.append(en)
        expected.append(de)

max_question_len = max(len(q) for q in questions)
max_expected_len = max(len(e) for e in expected)
questions = [' ' * (max_question_len - len(q)) + q for q in questions]
expected = [e + ' ' * (max_expected_len - len(e)) for e in expected]
if INVERT:
    questions = [q[::-1] for q in questions]

print('Total addition questions:', len(questions))

Total addition questions: 39929


In [109]:
chars = set(ch for k, v in zip(questions, expected) for ch in k + v)
ctable = CharacterTable(chars)
len(chars)

40

In [110]:
print('Vectorization...')
x = np.zeros((len(questions), max_question_len, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), max_expected_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, max_question_len)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, max_expected_len)
print('done')

Vectorization...
done


In [111]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Training Data:
(35937, 31, 40)
(35937, 32, 40)
Validation Data:
(3992, 31, 40)
(3992, 32, 40)


In [114]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(max_question_len, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
#model.add(layers.Dropout(DROP_OUT))
model.add(layers.RepeatVector(max_expected_len))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
#    model.add(layers.Dropout(DROP_OUT))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 128)               86528     
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 32, 128)           0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 32, 128)           131584    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 32, 40)            5160      
_________________________________________________________________
activation_14 (Activation)   (None, 32, 40)            0         
Total params: 223,272
Trainable params: 223,272
Non-trainable params: 0
_________________________________________________________________


In [122]:
def create_seq2seq(num_nodes, num_layers):
    question = Input(shape=(max_question_len, len(chars)), name='question')
    repeat = RepeatVector(max_expected_len)(question)
    prev = question
    for _ in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name='lstm_layer_%d' % (i + 1))(prev)
        prev = lstm
    dense = TimeDistributed(Dense(num_chars, name='dense', activation='softmax'))(prev)
    model = Model(inputs=[input], outputs=[dense])
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seq2seq = create_seq2seq(128, 1)

ValueError: Input 0 is incompatible with layer repeat_vector_16: expected ndim=2, found ndim=3

In [115]:

BATCH_SIZE = 2048

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print(q[::-1] if INVERT else q, '(%s)' % correct, '-', guess)


Train on 35937 samples, validate on 3992 samples
Epoch 1/10
35937/35937 [==============================] - 2s - loss: 2.2199 - acc: 0.6664 - val_loss: 1.3835 - val_acc: 0.7049
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 1.2799 - acc: 0.7063 - val_loss: 1.1512 - val_acc: 0.7049
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 1.0915 - acc: 0.7068 - val_loss: 1.0528 - val_acc: 0.7107
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 1.0313 - acc: 0.7146 - val_loss: 1.0126 - val_acc: 0.7182
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.9860 - acc: 0.7208 - val_loss: 0.9341 - val_acc: 0.7154
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.9130 - acc: 0.7275 - val_loss: 0.8892 - val_acc: 0.7381
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.8612 - acc: 0.7354 - val_loss: 0.8448 - val_acc: 0.7324
Epoch 8/10
35937/35937 [==============================] - 1s -

35937/35937 [==============================] - 1s - loss: 0.7669 - acc: 0.7665 - val_loss: 0.7745 - val_acc: 0.7641
Epoch 9/10
35937/35937 [==============================] - 1s - loss: 0.8141 - acc: 0.7482 - val_loss: 0.7933 - val_acc: 0.7545
Epoch 10/10
35937/35937 [==============================] - 1s - loss: 0.7798 - acc: 0.7616 - val_loss: 0.7724 - val_acc: 0.7639

--------------------------------------------------
Iteration 4
                       brummell (brummells                       ) - aaaaiiees                       
                       addendum (addendums                       ) - aaaaiiees                       
                     rakishness (rakishnesses                    ) - aaoooiiiies                     
                phyllocladaceae (phyllocladaceaes                ) - ceeooooiiiiiies                 
                        faddist (faddists                        ) - aaaiiees                        
                       penlight (penlights             

35937/35937 [==============================] - 1s - loss: 0.7618 - acc: 0.7641 - val_loss: 0.7624 - val_acc: 0.7641
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 0.7587 - acc: 0.7668 - val_loss: 0.7606 - val_acc: 0.7668
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 0.7566 - acc: 0.7683 - val_loss: 0.7584 - val_acc: 0.7677
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 0.7551 - acc: 0.7692 - val_loss: 0.7572 - val_acc: 0.7680
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.7540 - acc: 0.7695 - val_loss: 0.7558 - val_acc: 0.7681
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.7529 - acc: 0.7700 - val_loss: 0.7549 - val_acc: 0.7689
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.7518 - acc: 0.7704 - val_loss: 0.7538 - val_acc: 0.7689
Epoch 8/10
35937/35937 [==============================] - 1s - loss: 0.7507 - acc: 0.7706 - val_loss: 0.7528 - val_acc: 0.

35937/35937 [==============================] - 1s - loss: 0.7521 - acc: 0.7700 - val_loss: 0.7536 - val_acc: 0.7700
Epoch 9/10
35937/35937 [==============================] - 1s - loss: 0.7503 - acc: 0.7712 - val_loss: 0.7516 - val_acc: 0.7696
Epoch 10/10
35937/35937 [==============================] - 1s - loss: 0.7487 - acc: 0.7718 - val_loss: 0.7506 - val_acc: 0.7707

--------------------------------------------------
Iteration 11
                      soundness (soundnesses                     ) - carroiiiees                     
                         ordeal (ordeals                         ) - sariies                         
                       drepanis (drepani                         ) - saraiiies                       
                        sledder (sledders                        ) - sariiees                        
                  extravasation (extravasations                  ) - cerroooiiiiees                  
                           jong (jongs                

35937/35937 [==============================] - 1s - loss: 0.7298 - acc: 0.7769 - val_loss: 0.7301 - val_acc: 0.7762
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 0.7276 - acc: 0.7784 - val_loss: 0.7269 - val_acc: 0.7785
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 0.7224 - acc: 0.7807 - val_loss: 0.7252 - val_acc: 0.7781
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 0.7238 - acc: 0.7804 - val_loss: 0.7210 - val_acc: 0.7800
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.7194 - acc: 0.7820 - val_loss: 0.7187 - val_acc: 0.7814
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.7150 - acc: 0.7833 - val_loss: 0.7170 - val_acc: 0.7810
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.7136 - acc: 0.7835 - val_loss: 0.7148 - val_acc: 0.7821
Epoch 8/10
35937/35937 [==============================] - 1s - loss: 0.7631 - acc: 0.7670 - val_loss: 0.7456 - val_acc: 0.

...

35937/35937 [==============================] - 1s - loss: 0.0254 - acc: 0.9929 - val_loss: 0.0730 - val_acc: 0.9794
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 0.0276 - acc: 0.9919 - val_loss: 0.0712 - val_acc: 0.9799
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 0.0257 - acc: 0.9928 - val_loss: 0.0719 - val_acc: 0.9796
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 0.0252 - acc: 0.9930 - val_loss: 0.0708 - val_acc: 0.9803
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.0244 - acc: 0.9932 - val_loss: 0.0711 - val_acc: 0.9800
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.0241 - acc: 0.9933 - val_loss: 0.0715 - val_acc: 0.9798
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.0248 - acc: 0.9930 - val_loss: 0.0714 - val_acc: 0.9796
Epoch 8/10
35937/35937 [==============================] - 1s - loss: 0.0257 - acc: 0.9926 - val_loss: 0.0712 - val_acc: 0.

35937/35937 [==============================] - 1s - loss: 0.0417 - acc: 0.9875 - val_loss: 0.0794 - val_acc: 0.9771
Epoch 9/10
35937/35937 [==============================] - 1s - loss: 0.0412 - acc: 0.9877 - val_loss: 0.0801 - val_acc: 0.9771
Epoch 10/10
35937/35937 [==============================] - 1s - loss: 0.0407 - acc: 0.9880 - val_loss: 0.0795 - val_acc: 0.9773

--------------------------------------------------
Iteration 186
                       standing (standings                       ) - standings                       
                        zamboni (zambonis                        ) - zambonis                        
                      caledonia (caledonias                      ) - caledonias                      
                          polyp (polyps                          ) - polyps                          
                     solicitude (solicitudes                     ) - solicitudes                     
                         coleus (coleuses            

35937/35937 [==============================] - 1s - loss: 0.0291 - acc: 0.9919 - val_loss: 0.0715 - val_acc: 0.9803
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 0.0284 - acc: 0.9921 - val_loss: 0.0712 - val_acc: 0.9802
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 0.0282 - acc: 0.9922 - val_loss: 0.0710 - val_acc: 0.9802
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 0.0283 - acc: 0.9921 - val_loss: 0.0713 - val_acc: 0.9802
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.0280 - acc: 0.9922 - val_loss: 0.0717 - val_acc: 0.9803
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.0279 - acc: 0.9923 - val_loss: 0.0708 - val_acc: 0.9805
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.0274 - acc: 0.9925 - val_loss: 0.0710 - val_acc: 0.9804
Epoch 8/10
35937/35937 [==============================] - 1s - loss: 0.0278 - acc: 0.9924 - val_loss: 0.0718 - val_acc: 0.

35937/35937 [==============================] - 1s - loss: 0.0321 - acc: 0.9906 - val_loss: 0.0740 - val_acc: 0.9795
Epoch 9/10
35937/35937 [==============================] - 1s - loss: 0.0313 - acc: 0.9909 - val_loss: 0.0718 - val_acc: 0.9801
Epoch 10/10
35937/35937 [==============================] - 1s - loss: 0.0296 - acc: 0.9916 - val_loss: 0.0714 - val_acc: 0.9803

--------------------------------------------------
Iteration 193
                         cahoot (cahoots                         ) - cahoots                         
                   bilingualism (bilingualisms                   ) - bilingualists                   
                        aspirin (aspirins                        ) - aspirins                        
                           race (races                           ) - races                           
                     scoreboard (scoreboards                     ) - scoreboards                     
                        pompano (pompanoes           

35937/35937 [==============================] - 1s - loss: 0.0227 - acc: 0.9939 - val_loss: 0.0696 - val_acc: 0.9812
Epoch 2/10
35937/35937 [==============================] - 1s - loss: 0.0221 - acc: 0.9943 - val_loss: 0.0696 - val_acc: 0.9815
Epoch 3/10
35937/35937 [==============================] - 1s - loss: 0.0217 - acc: 0.9944 - val_loss: 0.0699 - val_acc: 0.9811
Epoch 4/10
35937/35937 [==============================] - 1s - loss: 0.0216 - acc: 0.9944 - val_loss: 0.0691 - val_acc: 0.9815
Epoch 5/10
35937/35937 [==============================] - 1s - loss: 0.0214 - acc: 0.9945 - val_loss: 0.0699 - val_acc: 0.9814
Epoch 6/10
35937/35937 [==============================] - 1s - loss: 0.0215 - acc: 0.9944 - val_loss: 0.0702 - val_acc: 0.9813
Epoch 7/10
35937/35937 [==============================] - 1s - loss: 0.0226 - acc: 0.9939 - val_loss: 0.0736 - val_acc: 0.9805
Epoch 8/10
35937/35937 [==============================] - 1s - loss: 0.1803 - acc: 0.9710 - val_loss: 0.6496 - val_acc: 0.

In [86]:
shakespeare = strip_headers(load_etext(100))
tokens = [tuple(word) for word in tokenize(plays, to_lower=True)]
token_counts = Counter(tokens)

In [90]:
pairs = [(token[i], token[i + 1], token_id) for token_id, token in enumerate(tokens) for i in range(len(token) - 1)]

In [91]:
pairs[10]

('n', 'd', 3)

In [82]:
help(tokenize)

Help on function tokenize in module gensim.utils:

tokenize(text, lowercase=False, deacc=False, errors='strict', to_lower=False, lower=False)
    Iteratively yield tokens as unicode strings, removing accent marks
    and optionally lowercasing the unidoce string by assigning True
    to one of the parameters, lowercase, to_lower, or lower.
    
    Input text may be either unicode or utf8-encoded byte string.
    
    The tokens on output are maximal contiguous sequences of alphabetic
    characters (no digits!).
    
    >>> list(tokenize('Nic nemůže letět rychlostí vyšší, než 300 tisíc kilometrů za sekundu!', deacc = True))
    [u'Nic', u'nemuze', u'letet', u'rychlosti', u'vyssi', u'nez', u'tisic', u'kilometru', u'za', u'sekundu']



In [87]:
tokens[5]

('b', 'y')